In [1]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
!pip install -U git+https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git

#Make sure you update it before starting this notebook
import lucem_illud #pip install -U git+https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git


#All these packages need to be installed from pip
#These are all for the cluster detection
import sklearn
import sklearn.feature_extraction.text
import sklearn.pipeline
import sklearn.preprocessing
import sklearn.datasets
import sklearn.cluster
import sklearn.decomposition
import sklearn.metrics

import scipy #For hierarchical clustering and some visuals
#import scipy.cluster.hierarchy
import gensim#For topic modeling
import requests #For downloading our datasets
import numpy as np #for arrays
import pandas as pd#gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import matplotlib.cm #Still for graphics
import seaborn as sns #Makes the graphics look nicer

#This 'magic' command makes the plots work better
#in the notebook, don't use it outside of a notebook.
#Also you can ignore the warning, it
%matplotlib inline

import itertools
import json

  Cloning https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git to /tmp/pip-req-build-ar_cck_h
  Running command git clone --filter=blob:none --quiet https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git /tmp/pip-req-build-ar_cck_h
  Resolved https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git to commit b17a265d3b8253424e5b38872457f7437909a65d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.8/76.8 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
df = pd.read_excel('/content/drive/MyDrive/MA_Thesis/DBCommunity/saved_data/reply_raw_data.xlsx')
df.columns

Index(['字段1', '字段2', '标题', '标题链接', 'bg-img-green', 'reply-content', 'pubtime',
       'comment-vote'],
      dtype='object')

In [22]:
df.rename(columns={'Unnamed: 0': 'id', '字段1': 'Title', '字段2': 'Author', '标题': 'Reply_author', '标题链接': 'reply_author_id', 'reply-content': 'reply_content', 'pubtime': 'reply_pubtime', 'comment-vote': 'comment_like'}, inplace=True)
df.drop(columns=['bg-img-green'], inplace=True)

df.head()

,Title,Author,Reply_author,reply_author_id,reply_content,reply_pubtime,comment_like
0,小组活动｜🎉小组官方slack群建立啦!🎉,Anon加重音,Wollstonecraft,https://www.douban.com/people/154742699/,支持！,2020-10-09 08:46:52,赞 (7)
1,小组活动｜🎉小组官方slack群建立啦!🎉,Anon加重音,树懒,https://www.douban.com/people/xtopia/,dd支持,2020-10-09 09:16:52,赞
2,小组活动｜🎉小组官方slack群建立啦!🎉,Anon加重音,小小脑袋,https://www.douban.com/people/149862817/,dd支持,2020-10-09 09:22:28,赞
3,小组活动｜🎉小组官方slack群建立啦!🎉,Anon加重音,沐沫沫,https://www.douban.com/people/197316067/,加入啦！,2020-10-09 09:24:35,赞
4,小组活动｜🎉小组官方slack群建立啦!🎉,Anon加重音,かん,https://www.douban.com/people/183851723/,dd,2020-10-09 09:29:19,赞


In [23]:
# prompt: modify column "reply_author_id" with regular expression, extracting only the content between "https://www.douban.com/people/" and "/" in each string

import re

def extract_id(url):
  match = re.search(r"https://www.douban.com/people/([^/]+)/", url)
  if match:
    return match.group(1)
  else:
    return None

df['reply_author_id'] = df['reply_author_id'].apply(extract_id)
df['reply_author_id']

,reply_author_id
0,154742699
1,xtopia
2,149862817
3,197316067
4,183851723
...,...
13652,70453945
13653,151153332
13654,70453945
13655,221863224


In [24]:
# prompt: modify column "comment_like" with regular expression, extracting only the number between "(" and ")". If there's no such expression, mark 0 in that row.

def extract_number(text):
  match = re.search(r'\((\d+)\)', str(text))
  if match:
    return int(match.group(1))
  else:
    return 0

df['comment_like'] = df['comment_like'].apply(extract_number)
df['comment_like']

,comment_like
0,7
1,0
2,0
3,0
4,0
...,...
13652,1
13653,1
13654,1
13655,3


In [26]:
# Ensure the column is in string format
df['reply_pubtime'] = df['reply_pubtime'].astype(str)

# Clean the "reply_pubtime" column
df_pubtime = df['reply_pubtime'].str.extract(r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})')[0]

df_pubtime.tail()

,0
13652,2023-07-29 14:41:57
13653,2023-07-29 16:53:17
13654,2023-07-29 17:16:58
13655,2023-08-03 05:43:57
13656,2023-08-03 08:41:36


In [28]:
# prompt: convert df_pubtime to time format

import pandas as pd

# Assuming df_pubtime is already defined as in your provided code
# ... (your existing code to create df_pubtime) ...

# Convert the 'df_pubtime' Series to datetime objects
df['reply_pubtime'] = pd.to_datetime(df_pubtime, errors='coerce')

# Now 'df['reply_pubtime']' contains datetime objects.
# You can handle errors (invalid date strings) with errors='coerce' which sets invalid dates to NaT.
# Check for NaT values if needed:
# df[df['reply_pubtime'].isna()]

df.head()

,Title,Author,Reply_author,reply_author_id,reply_content,reply_pubtime,comment_like
0,小组活动｜🎉小组官方slack群建立啦!🎉,Anon加重音,Wollstonecraft,154742699,支持！,2020-10-09 08:46:52,7
1,小组活动｜🎉小组官方slack群建立啦!🎉,Anon加重音,树懒,xtopia,dd支持,2020-10-09 09:16:52,0
2,小组活动｜🎉小组官方slack群建立啦!🎉,Anon加重音,小小脑袋,149862817,dd支持,2020-10-09 09:22:28,0
3,小组活动｜🎉小组官方slack群建立啦!🎉,Anon加重音,沐沫沫,197316067,加入啦！,2020-10-09 09:24:35,0
4,小组活动｜🎉小组官方slack群建立啦!🎉,Anon加重音,かん,183851723,dd,2020-10-09 09:29:19,0


In [29]:
# prompt: print missing value in reply_pubtime

# Find rows where 'reply_pubtime' is missing (NaT)
missing_pubtime = df[df['reply_pubtime'].isna()]

# Print the rows with missing 'reply_pubtime'
missing_pubtime

,Title,Author,Reply_author,reply_author_id,reply_content,reply_pubtime,comment_like


## Identify active users

In [31]:
# prompt: calculate how many times every unique "reply_author" has posted, and create a new dataframe called author_reply_counts, with a column called "reply_author" and a colomn is "Reply_Count"

author_reply_counts = df.groupby('Reply_author')['Reply_author'].count().reset_index(name='Reply_Count')
author_reply_counts

,Reply_author,Reply_Count
0,#Elysium#,1
1,#第三只耳朵#,3
2,#鼓鼓#,1
3,(˃ ⌑ ˂ഃ ),2
4,(ﾟoﾟ;,5
...,...,...
3620,🦞,1
3621,🧊,2
3622,🧣,5
3623,🪸,1


In [34]:
# prompt: save the dataset as "author_reply_counts.csv"

author_reply_counts.to_csv('/content/drive/MyDrive/MA_Thesis/DBCommunity/saved_data/author_reply_counts.csv', index=False)

In [35]:
# prompt: save df as "reply_aca.csv"

df.to_csv('/content/drive/MyDrive/MA_Thesis/DBCommunity/saved_data/reply_aca.csv', index=False)